<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Relational-Database-and-SQL" data-toc-modified-id="Relational-Database-and-SQL-1">Relational Database and SQL</a></span><ul class="toc-item"><li><span><a href="#Relational-Model" data-toc-modified-id="Relational-Model-1.1">Relational Model</a></span></li><li><span><a href="#Structured-Query-Language-(SQL)" data-toc-modified-id="Structured-Query-Language-(SQL)-1.2">Structured Query Language (SQL)</a></span></li></ul></li></ul></div>

In [1]:
# install packages
# !pip install nbinteract

In [2]:
# HIDDEN

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import nbinteract as nbi
import sqlalchemy


sns.set()
sns.set_context('talk')
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8

# Relational Database and SQL

**Database** is an organized collection of data; **Database Management System (DBMS)** is a software
system that stores, manages, and facilitates access to one or more databases. While they are two different things, the term "database" is sometimes loosely used to refer to DBMS. 

DBMS is useful for many reasons. It provides reliable storage to survive system crashes and disk failures. It organizes data and enforces consistent properties (e.g. some values can't be negative). It also protects data by controlling data access, and provides efficient access with optimized data structures and optimized computation for user queries.

There are a variety of different DBMSs, but the most common one is **relational database management system(RDBMS)**, which organizes data by relational models.  

## Relational Model

**Relation** means table, which describes some relationships between data. In the **relational model** paradigm, DBMSs logically organize data into relations. Each relation is strictly organized by a **schema**, which defines data types, constraints, and table columns. The specific data records in a table are called **instances**, and they must satisfy the corresponding schema. 

In a table, these terms are synonyms: **tuple** = **record** = **row**; **attribute** = **column**. From the graph we can see that a row refers to a data instance, and a column refers to part of an instance.

![relation](https://github.com/weiwzhang/writings/blob/master/simplistic_SQL_tutorial_ds100/relation.png)

**Structured Query Language (SQL)** is used to define, manipulat and compute on data in a RDBMS. 

Relational model is widely used in the data world for its many benefits. For example, tabular format is easy to perform computations on and also shows relationships between data. schemas can enforce data constraints. SQL allows you to write simple and powerful data queries and computations (see following section).


## Structured Query Language (SQL)


SQL is one of the most common languages used in data systems. It's a **declarative language**, which means you define the output you want to get in your **query**, and let the DBMS figure out how to best execute your request. More specifically, the query optimizer in DBMS generates many possible plans and choose the most efficient to execute the query. 

SQL is useful because it's more intuitive to understand for non-programmers. You don't need to know how to best execute your query; you just need to know what output you want. It can also be more efficient because DBMS optimizes query executions. However, the system performance might heavily depend on the how well the DBMS can optimize queries. 

SQL has two sublanguages: **Data Definition Language** (DDL) which modifies schema, and **Data Manipulation Language **(DML) is what we commonly use to write queries. So the following SQL code are all DML. 

So, how do we use SQL? We'll introduce some most basic SQL query keywords through an example. For an overview here are the words we'll cover:
- SELECT
- FROM
- WHERE
- LIMIT
- GROUP BY and Aggregations
- other useful keywords: HAVING, ORDER BY, LIMIT

(Note: we will use SQLite, which is a simple relational DBMS contained in a C programming library. Here we launch a SQLite database stored in the data100.db file in the local folder.)

In [3]:
# Delete the database if it already exists.
from pathlib import Path
dbfile = Path("data100.db")
if dbfile.exists():
    dbfile.unlink()

sqlite_uri = "sqlite:///data100.db"
sqlite_engine = sqlalchemy.create_engine(sqlite_uri)

#### A Data Science Carnival

Berkeley is hosting the first ever Data Science Carnival, and you are taking your little sister to go and explore the magnificent fun of data science. Your sister wants to get a colorful balloon. To help your sister out, you want to find the dataset for all balloons, and then choose the ones your sister wants. Here suppose that each balloon type is uniquely identified by an id and has the following information: a shape, a color, the clown who sells it, and the cost of this particular balloon type.

To create a dataset, first let's introduce the **`CREATE TABLE`** clause for creating schemas:

```sql
CREATE TABLE tablename (
    col1_name data_type1, 
    col2_name data_type2, 
    ...
);
```

In [4]:
# create tables in Pandas
sql_expr = """
CREATE TABLE Balloons (
    bid INT,
    bshape text,
    bcolor text,
    clown text,
    cost float,
    PRIMARY KEY (bid)
);
"""

result = sqlite_engine.execute(sql_expr)

Here we introduce the idea of **primary key**, which is an attribute that uniquely identifies a row in a table. It's useful because it enforces unique records in a table. In some more complicated tables, we might have one or more columns referring to primary key columns in other tables. In this case, these columns are called foreign keys. A **foreign key** is one or more columns that uniquely identifies a row of another table or the same table. When we introduce `JOIN` in next section, we might see more cases of foreign keys. 

Now we can add data data records into these tables using **`INSERT VALUE`** clause: 

```sql
INSERT INTO tablename 
    (col1, col2, ...) 
VALUES 
    (v11, v12, ...), 
    (v21, v22, ...), 
    ...
;
```

In [5]:
sql_expr = """
INSERT INTO Balloons
    (bid, bshape, bcolor, clown, cost)
VALUES
    (1, 'rectangular prism', 'red', 'Whompers LeFou', 1.5),
    (2, 'ellipsoid', 'red', 'Cayenne East', 2.0),
    (3, 'hyperboloid', 'red', 'Swan Ronson', 8.9),
    (4, 'rectangular prism', 'green', 'Whompers LeFou', 1.5),
    (5, 'ellipsoid', 'red', 'Cayenne East', 1.5),
    (6, 'octahedral prism', 'white', 'Swan Ronson', 2.0),
    (7, 'octahedral prism', 'green', 'Swan Ronson', 3.0),
    (8, 'extremely rare shape', 'extremely rare color', 'Reese Withoutaspoon', 3.5)
;
"""

result = sqlite_engine.execute(sql_expr)

Ok, now that we have the dataset, let's inspect the data and query for what we need!

### SELECT... FROM

Your sister wants to know what colors are available for the balloons. How do we get that? We notice that all the colors are listed in the `Balloons` table, so we will **`SELECT`** the colors **`FROM`** `Balloons`. `SELECT` allows us to choose one or more specific columns `FROM` one or more tables (see `Joins` in later sections). 

```sql
SELECT col
FROM table;
```

In [6]:
sql_expr = """
SELECT DISTINCT bcolor FROM Balloons;
"""
pd.read_sql(sql_expr, sqlite_engine)

,bcolor
0,red
1,green
2,white
3,extremely rare color


Note: here we introduced another new keyword, **`DISTINCT`**, which basically filters out duplicates and only leave unique values in the output. For example here after selecting all the colors, `DISTINCT` will filter out duplicates and leave four final unique colors in the output. 

`SELECT... FROM` is the simplest format of a query. All other keywords below are not mandatory syntax but useful for different kinds of data requests.

### WHERE

Your sister prefers the color red over other colors. How do we filter out only red balloons? This is where **`WHERE`** keyword becomes useful: it allows you to filter out specific rows in the final output according to some column values. 

```sql
SELECT cols
FROM tables
[WHERE conditions];
```

In [7]:
sql_expr = """
SELECT *
FROM Balloons
WHERE bcolor = 'red';
"""
pd.read_sql(sql_expr, sqlite_engine)

,bid,bshape,bcolor,clown,cost
0,1,rectangular prism,red,Whompers LeFou,1.5
1,2,ellipsoid,red,Cayenne East,2.0
2,3,hyperboloid,red,Swan Ronson,8.9
3,5,ellipsoid,red,Cayenne East,1.5


Note that here we use **`*`**, which means selecting all columns in a record. 

### GROUP BY and Aggregations

Now let's do some more complicated calculations. For example, you might be curious about the average price for each color of balloons. How do we solve this problem? 

We actually already know how to do this using pandas. This is probably how you might approach this problem using pandas (suppose we have a `balloons` dataframe):

In [8]:
# create the dataframe balloons
balloons = pd.read_sql("""SELECT * FROM Balloons;""", sqlite_engine)
balloons

,bid,bshape,bcolor,clown,cost
0,1,rectangular prism,red,Whompers LeFou,1.5
1,2,ellipsoid,red,Cayenne East,2.0
2,3,hyperboloid,red,Swan Ronson,8.9
...,...,...,...,...,...
5,6,octahedral prism,white,Swan Ronson,2.0
6,7,octahedral prism,green,Swan Ronson,3.0
7,8,extremely rare shape,extremely rare color,Reese Withoutaspoon,3.5


In [9]:
balloons.groupby('bcolor')['cost'].mean()

bcolor
extremely rare color    3.500
green                   2.250
red                     3.475
white                   2.000
Name: cost, dtype: float64

If we analyze the above approach, we can see that there are two steps: first, form groups of balloons by different colors; second, calculate the mean cost in each group.

SQL **`GROUP BY`** is no different from pandas `groupby` in terms of clustering records into groups by certain criteria. After grouping, we can apply aggregation functions if we want to get more statistics on the groups, like the mean of a group of numbers. **Aggregation** functions take in a set of records, perform some computations, and return a single value. For example, if we want to find the average price, we can use `AVG(col)` function. 

```sql
SELECT ...
FROM ...
[WHERE ...]
[GROUP BY ...];
```

In [10]:
sql_expr = """
SELECT bcolor, AVG(cost)
FROM Balloons
GROUP BY bcolor;
"""
pd.read_sql(sql_expr, sqlite_engine)

,bcolor,AVG(cost)
0,extremely rare color,3.500
1,green,2.250
2,red,3.475
3,white,2.000


Note: `GROUP BY` and aggregation usually come in pairs, because it's common to first group values into different categories and then try to aggregate and get insights on each category.

### Other Keywords: HAVING, ORDER BY, LIMIT

`HAVING`, `ORDER BY` and `LIMIT` are three additional useful queries. **`HAVING`** is used with `GROUP BY` and is used to filter groups. **`ORDER BY`** orders the output records by certain criteria. **`LIMIT`** is a keyword that limits the number of output records. 

Going back to our example dataset, if I want to find bigger sellers, i.e. clowns who sell more than one type of balloons, and their average balloon prices, I might want to write a query like this:

In [11]:
sql_expr = """
SELECT clown, AVG(cost)
FROM Balloons
GROUP BY clown
HAVING COUNT(clown) > 1;
"""
pd.read_sql(sql_expr, sqlite_engine)

,clown,AVG(cost)
0,Cayenne East,1.750000
1,Swan Ronson,4.633333
2,Whompers LeFou,1.500000


Note that we use a new function, `COUNT`, which counts the number of records in the output (or in this case, in each group). 

Maybe we want out output from previous query to be ordered by average cost from high to low, then we can revise previous query:

In [12]:
sql_expr = """
SELECT clown, AVG(cost)
FROM Balloons
GROUP BY clown
HAVING COUNT(clown) > 1
ORDER BY AVG(cost) DESC;
"""
pd.read_sql(sql_expr, sqlite_engine)

,clown,AVG(cost)
0,Swan Ronson,4.633333
1,Cayenne East,1.750000
2,Whompers LeFou,1.500000


Note that we use a new keyword, `DESC` here, which just means to organize output numbers by descending order.

Suppose I only want to see the top two clowns who have the highest average prices, I can use `LIMIT` to limit my output records to only two. 

In [13]:
sql_expr = """
SELECT clown, AVG(cost)
FROM Balloons
GROUP BY clown
HAVING COUNT(clown) > 1
ORDER BY AVG(cost) DESC
LIMIT 2;
"""
pd.read_sql(sql_expr, sqlite_engine)

,clown,AVG(cost)
0,Swan Ronson,4.633333
1,Cayenne East,1.750000


#### Some Extra Notes on SQL:

Here we have used the most basic SQL queries, and hopefully with these skills you can easily find the right balloon(s) for your sister. To add a couple additional notes on using SQL:
- capitalization and indentation don't effect the correctness of query. So if you write `select` instead of `SELECT` query would run fine. 
- however order of execution is important in SQL. The following diagram from class illustrate the correct execution order between keywords:

![query_exec_order.png](https://github.com/weiwzhang/writings/blob/master/simplistic_SQL_tutorial_ds100/query_exec_order.png)

This probably also answers your confusion of why we need `HAVING` after we already have `WHERE`. It's because we execute `WHERE` first, and then we execute `GROUP BY` and `HAVING`, so we need `HAVING` if we want to filter groups.

So for final reference, a typical query structure would look like this:

```sql
SELECT ...
FROM ...
[WHERE ...]
[GROUP BY ...]
[HAVING ...]
[ORDER BY ...]
[LIMIT ...];
```

### Summary 

Today we get an overview of the database management system and relational models. We've also learned what SQL is and some most basic SQL queries. You might have some additional questions about:
- other DBMS types: NoSQL is a popular one. We won't cover it here but it's worth checking out
- more complicated SQL queries: there are other useful keywords (e.g. `AS`, `AND`), and we'll move on to `JOIN` next time which interacts with multiple tables. 